In [44]:
import pandas as pd
import os
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [56]:
df = None
folderpath = "../../Data/QMEF/ILIAS_Elon_Musk_Tweets/"

In [57]:
# Some tweets are longer than one line. The line break character \n gets misinterpreted as new tweet. Replace all \n with white space but the first one.
for filename in os.listdir(folderpath):
    with open(os.path.join(folderpath,filename), 'r', encoding='latin1') as file:
        before, first, after = file.read().partition('\n')
        result = before + first + after.replace('\n', ' ')
    with open(os.path.join(folderpath,filename), 'w', encoding='latin1') as file:
        file.write(result)

In [58]:
# Interate through all tweets and append them to a pandas DataFrame
# Could use further optimization - Runtime 10 seconds.
for filename in os.listdir(folderpath):
    df = pd.concat([df, pd.read_csv(os.path.join(folderpath,filename), encoding='utf8', sep='\t')], axis=0)

In [59]:
# Correcting the Data Types
df['UTC'] = pd.to_datetime(df['UTC'])
df['CreatedAt'] = pd.to_datetime(df['CreatedAt'])
df.dtypes

TweetId                     int64
Text                       object
Name                       object
ScreenName                 object
UTC           datetime64[ns, UTC]
CreatedAt     datetime64[ns, UTC]
Favorites                   int64
Retweets                    int64
Language                   object
Client                     object
TweetType                  object
URLs                       object
Hashtags                    int64
Mentions                    int64
MediaType                  object
MediaURLs                  object
dtype: object

In [60]:
analyzer = SentimentIntensityAnalyzer()

def get_sentiment(text):
    vs = analyzer.polarity_scores(text)
    return [vs['neg'], vs['pos'], vs['neu'], vs['compound']]

df[['neg', 'pos', 'neu', 'compound']] = df.apply(lambda x: get_sentiment(str(x['Text'])),result_type='expand', axis=1)

In [61]:
keywords = ['dogie', 'crypto', 'cryptocurrency', 'cryptocurrencies', 'bitcoin', 'etherium', 'coin', 'stonks', 'invest']

def get_crypto_boolean(text):
    text = text.lower()
    if any(word in text for word in keywords):
        return True
    return False

df['crypto related'] = df.apply(lambda x: get_crypto_boolean(str(x['Text'])), result_type='expand', axis = 1)

In [62]:
df.to_csv("../../Data/QMEF/tweet_analysis.csv", sep=";", encoding='utf8')
# Encoding führt zu Problemen.

In [65]:
df[['TweetId', 'Text', 'UTC', 'Favorites', 'Retweets', 'pos', 'neg', 'neu', 'compound', 'crypto related']]

,TweetId,Text,UTC,Favorites,Retweets,pos,neg,neu,compound,crypto related
0,1321513255332941824,@MachinePix Little crane: “Am I just a toy to ...,2020-10-28 18:04:47+00:00,1762,52,0.000,0.0,1.000,0.0000,False
0,1331745833629470722,@EvaFoxU We will have special colors for new R...,2020-11-25 23:45:24+00:00,3820,143,0.278,0.0,0.722,0.6124,False
0,1349459692515434502,@NASASpaceflight Sea shanty tiktok takes it to...,2021-01-13 20:53:57+00:00,4237,140,0.000,0.0,1.000,0.0000,False
0,1390920049372897281,@ericrihlmann @Tesla @nbcsnl @TimesSquareNYC @...,2021-05-08 06:42:36+00:00,13486,480,0.406,0.0,0.594,0.6249,False
0,1390839584225955847,Cybertruck prototype in New York this weekend,2021-05-08 01:22:52+00:00,272338,16591,0.000,0.0,1.000,0.0000,False
...,...,...,...,...,...,...,...,...,...,...
0,1391164130204192771,@flcnhvy Might still need a VPN in some countries,2021-05-08 22:52:29+00:00,11331,659,0.000,0.0,1.000,0.0000,False
0,1391130755787599881,Special link to view SNL outside USA https://t...,2021-05-08 20:39:52+00:00,175011,27144,0.278,0.0,0.722,0.4019,False
0,1390955387654914052,First time a Falcon rocket booster will reach ...,2021-05-08 09:03:01+00:00,95974,6651,0.084,0.0,0.916,0.0258,False
0,1390929314355548165,@WholeMarsBlog Yup 🤣🤣,2021-05-08 07:19:25+00:00,9159,538,0.390,0.0,0.610,0.7506,False
